In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix

np.random.seed(42)  # You can use any integer value
random_state = 42

In [34]:
modelNames = ["WW_redReward0_grReward1_7x7_300000steps",
                  "WW_redReward0_grReward1_7x7_1500000steps",
                  "WW_redReward1_grReward0_7x7_700000steps",
                  "WW_redReward1_grReward0_7x7_1500000steps",
                  "WW_redReward1_grReward0_7x7_2000000steps"]

red0green1_30k = pd.read_csv("C:/Projects/public/XAI_Master/datasets/WW_redReward0_grReward1_7x7_300000steps.csv",
    header=0,
    index_col=None,
    float_precision='high'
    )
red0green1_150k = pd.read_csv("C:/Projects/public/XAI_Master/datasets/WW_redReward0_grReward1_7x7_1500000steps.csv",
    header=0,
    index_col=None,
    float_precision='high'
    )
red1green0_70k = pd.read_csv("C:/Projects/public/XAI_Master/datasets/WW_redReward1_grReward0_7x7_700000steps.csv",
    header=0,          # Use first row as headers
    index_col=None,    # Don't use any column as index
    float_precision='high'  # For precise float values
    )
red1green0_150k = pd.read_csv("C:/Projects/public/XAI_Master/datasets/WW_redReward1_grReward0_7x7_1500000steps.csv",
    header=0,          # Use first row as headers
    index_col=None,    # Don't use any column as index
    float_precision='high'  # For precise float values
    )
red1green0_200k = pd.read_csv("C:/Projects/public/XAI_Master/datasets/WW_redReward1_grReward0_7x7_2000000steps.csv",
    header=0,          # Use first row as headers
    index_col=None,    # Don't use any column as index
    float_precision='high'  # For precise float values
    )


# Merge and shuffle the two datasets
df = pd.concat([red0green1_30k, red0green1_150k, red1green0_70k, red1green0_150k, red1green0_200k], ignore_index=True)
df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)
"""
df = red0green1
"""
x = df.drop('target', axis=1)
x = df.drop(['target',"agentX","agentY","redX","redY","greenX","greenY"], axis=1)
print(x.shape)
y = df['target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=random_state)

(50000, 128)


In [35]:
# Train and make predictions
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

# Print float results
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred):.4f}")
print(f"R2 Score: {r2_score(y_test, y_pred):.4f}")

# Round y_pred to int, max 1 and min -1
y_pred = np.clip(np.round(y_pred), -1, 1)

# Print int results
total_correct = (y_test == y_pred).sum()
total_samples = len(y_test)
print(f"Total correct predictions: {total_correct} out of {total_samples}")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2%}")
print(f"\nConfusion matrix:\n{confusion_matrix(y_test, y_pred)}")


Mean Squared Error: 0.0267
R2 Score: 0.9657
Total correct predictions: 9708 out of 10000
Accuracy: 97.08%

Confusion matrix:
[[5947   41    0]
 [  16 1082  112]
 [   0  123 2679]]


In [36]:
# Train and make predictions
tree = DecisionTreeClassifier(random_state=42)
tree.fit(x_train, y_train)
y_pred = tree.predict(x_test)

# Print results
total_correct = (y_test == y_pred).sum()
total_samples = len(y_test)
print(f"Total correct predictions: {total_correct} out of {total_samples}")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2%}")
print(f"\nConfusion matrix:\n{confusion_matrix(y_test, y_pred)}")

Total correct predictions: 9994 out of 10000
Accuracy: 99.94%

Confusion matrix:
[[5987    1    0]
 [   1 1208    1]
 [   0    3 2799]]


A baseline is established. A bit too good baseline. My thoughts now is that I wonder how much the decision tree learns to identify the neural net and therefore knows if it is a good or bad neural net. I'm thinking that I need many more neural nets to balance out this, especially very diffferent ones.

Todo:
- Train many models, 30+
- Create shap datasets

Questions:
- Should the models retain the 5% epsilon during dataset creation?
- Should 